In [1]:
home_dir = "/home/matteo"
import sys
project_dir = home_dir + "/git/real"
sys.path.append(project_dir)

import os
os.chdir("/home/matteo/git/real/")

import real.DataBaseUtils as dbu
from real.core.db_to_model import from_db_to_model_all
from real.core.convert_models import convert_to_gpu_model
from real.core.Model import Model
from real.core.GpuModel import GpuModel
from real.core.exhaustive_calibration import call_one_simu_cupy
from real.NameUtils import State
from real.config.config import project_path
from real.core.convert_models import convert_simu_to_vdata
import pickle
from real.core.vdata_utils import reference_vdata_to_db, vdata_from_filename, export_to_monocle_csv
import numpy as np
from real.Utils import md5
import vdata

db = dbu.connect_to_database()

dump_dir = "3-RetroEngineering/1-REAL_insilico_validation"

In [2]:
id_network = 553
id_simProtocol = 28
seed = 666

model:Model = from_db_to_model_all(db=db, id_sim_protocol=id_simProtocol, id_grn=id_network, seed=seed)
gpu_model:GpuModel = convert_to_gpu_model(model)

"""
gpu_model.sim_protocol.record.list_state_to_record = np.array([State.get_index(State.RNA),
                                                               State.get_index(State.CP),
                                                               State.get_index(State.KON),
                                                               State.get_index(State.KOFF)])
plot.plot_data_4main_from_model(gpu_model, rec_data_gpu, ['A', 'B', 'D', 'E'])
import matplotlib.pyplot as plt
plt.show()
"""
gpu_model.sim_protocol.record.list_state_to_record = np.array([State.get_index(State.RNA),
                                                               State.get_index(State.CP)])

gpu_model.sim_protocol.record.n_state_to_record = len(gpu_model.sim_protocol.record.list_state_to_record)

# specify only time point to record
#TimePoints = [-40, -20, 0, 5, 10, 15, 20, 25, 30, 50, 90, 100, 120, 140, 160, 200]
#TimePoints = [-40, 0, 5, 10, 15, 20, 30, 50, 100, 150]
TimePoints = [-40,-20,0,5,10,15,20,25,30,50,90,100]
#TimePoints = [-40,0,5,10,15,20,25,30,50,90,100]
#TimePoints = [0,5,10,15,20,25,30,50,90,100]

gpu_model.sim_protocol.set_time_point_to_record(TimePoints)

rec_data_gpu, kox_beta = call_one_simu_cupy(gpu_model)

In [3]:
vsimu = convert_simu_to_vdata(rec_data_gpu, gpu_model)[0]
vsimu.name = f'insilico simulation for grn {id_network}'

In [4]:
vsimu.var

,gene_ensembl_id,gene_shortname
0,A,A
1,B,B
2,C,C
3,D,D
4,E,E
5,F,F
6,G,G
7,G1,G1
8,G10,G10
9,G2,G2


In [5]:
# Add cell_type
vsimu.obs.insert(1,'cell_type','WT')

# Change cell_index name to add cell_type (mandatory for concatenation)
vsimu.set_obs_index([str(i)+'_WT' for i in vsimu.obs.index])

# Change genes' index to set as genes' name (from Ensembl_Gene_ID
vsimu.set_var_index(vsimu.var.gene_ensembl_id.values)

In [6]:
vsimu.var

,gene_ensembl_id,gene_shortname
A,A,A
B,B,B
C,C,C
D,D,D
E,E,E
F,F,F
G,G,G
G1,G1,G1
G10,G10,G10
G2,G2,G2


In [7]:
vdata.setLoggingLevel('INFO')

dump_record_dir = project_path / dump_dir / 'generated_insilico_data'
dump_filename = dump_record_dir / f'record_data_Network_{id_network}_200_cells_matteo_is_testing.h5'
vsimu.write(dump_filename)
vsimu.file.close()

12:07:55 :       INFO : [write.py] Saving dict layers
12:07:55 :       INFO : [write.py]   ↳ Saving TemporalDataFrame mRNA
12:07:55 :       INFO : [write.py]     ↳ Saving Series index
12:07:55 :       INFO : [write.py]     ↳ Saving None value for time_col_name
12:07:55 :       INFO : [write.py]     ↳ Saving Series columns
12:07:55 :       INFO : [write.py]     ↳ Saving time point -40.0h
12:07:55 :       INFO : [write.py]     ↳ Saving time point -20.0h
12:07:55 :       INFO : [write.py]     ↳ Saving time point 0.0h
12:07:55 :       INFO : [write.py]     ↳ Saving time point 5.0h
12:07:55 :       INFO : [write.py]     ↳ Saving time point 10.0h
12:07:55 :       INFO : [write.py]     ↳ Saving time point 15.0h
12:07:55 :       INFO : [write.py]     ↳ Saving time point 20.0h
12:07:55 :       INFO : [write.py]     ↳ Saving time point 25.0h
12:07:55 :       INFO : [write.py]     ↳ Saving time point 30.0h
12:07:55 :       INFO : [write.py]     ↳ Saving time point 50.0h
12:07:55 :       INFO : [w

In [12]:
# load back in
vsimu_back = vdata.read(dump_filename)

12:08:55 :       INFO : [read.py] Got key : 'layers'.
12:08:55 :       INFO : [read.py]   ↳ Reading dict /layers.
12:08:55 :       INFO : [read.py]     ↳ Reading chunked TemporalDataFrame /layers/Protein_C.
12:08:55 :       INFO : [read.py]       ↳ Reading Series /layers/Protein_C/columns.
12:08:55 :       INFO : [read.py]         ↳ Reading array /layers/Protein_C/columns.
12:08:55 :       INFO : [read.py]       ↳ Reading Series /layers/Protein_C/index.
12:08:55 :       INFO : [read.py]         ↳ Reading array /layers/Protein_C/index.
12:08:55 :       INFO : [read.py]       ↳ Reading None.
12:08:55 :       INFO : [read.py]     ↳ Reading chunked TemporalDataFrame /layers/mRNA.
12:08:55 :       INFO : [read.py]       ↳ Reading Series /layers/mRNA/columns.
12:08:55 :       INFO : [read.py]         ↳ Reading array /layers/mRNA/columns.
12:08:55 :       INFO : [read.py]       ↳ Reading Series /layers/mRNA/index.
12:08:55 :       INFO : [read.py]         ↳ Reading array /layers/mRNA/index.
1

In [13]:
vsimu_back.var

,gene_ensembl_id,gene_shortname
A,NaN,NaN
B,NaN,NaN
C,NaN,NaN
D,NaN,NaN
E,NaN,NaN
F,NaN,NaN
G,NaN,NaN
G1,NaN,NaN
G10,NaN,NaN
G2,NaN,NaN


In [14]:
vsimu_back.file.close()